In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install rouge-score py7zr
!pip install transformers
!pip install datasets
!pip install tokenizers
!pip install evaluate
!pip install --upgrade accelerate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 69.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=8bb70efffca6ee78d348a0c90241661b6804082205bd0558a1186dce8bca704f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.5 MB/s eta 0:00:00
     ━

In [2]:
!python -c "import nltk; nltk.download('punkt');"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
%cp -av /kaggle/input/season-samsum/* /kaggle/working/

'/kaggle/input/season-samsum/model.py' -> '/kaggle/working/model.py'
'/kaggle/input/season-samsum/run.py' -> '/kaggle/working/run.py'
'/kaggle/input/season-samsum/trainer.py' -> '/kaggle/working/trainer.py'


In [4]:
mkdir data

In [5]:
import json
from typing import Any
from functools import partial

import datasets
from transformers import PreTrainedTokenizerBase
from nltk import sent_tokenize
from rouge_score import rouge_scorer
from transformers import AutoTokenizer

dataset = datasets.load_dataset('samsum', split=["train[:5000]","validation[:625]","test[:625]"])
dataset = datasets.DatasetDict({'train':dataset[0],'validation':dataset[1],'test':dataset[2]})
src_text_column_name, tgt_text_column_name = "dialogue", "summary"
max_source_length, max_target_length = 512, 128
n_proc = 40

# Since bos_token is used as the beginning of target sequence,
# we use mask_token to represent the beginning of each sentence.
bosent_token = "<mask>"
bosent_token_id = 50264

rouge_scorer = rouge_scorer.RougeScorer(['rougeLsum'], use_stemmer=True)


def convert_to_features(
        examples: Any,
        tokenizer: PreTrainedTokenizerBase,
        padding: str,
        max_source_length: int,
        max_target_length: int,
        src_text_column_name: str,
        tgt_text_column_name: str,
):
    inputs, targets = [], []
    all_sent_rouge_scores = []
    for i in range(len(examples[src_text_column_name])):
        if examples[src_text_column_name][i] is not None and examples[tgt_text_column_name][i] is not None:
            input_sentences = sent_tokenize(examples[src_text_column_name][i])
            target_sentences = examples[tgt_text_column_name][i].strip()
            rouge_scores = []
            for sent in input_sentences:
                rouge_scores.append(rouge_scorer.score(target_sentences, sent)['rougeLsum'].fmeasure)
            # todo: add bos_token this way is unsafe
            inputs.append(bosent_token.join(input_sentences))
            targets.append(target_sentences.replace('\n', ' ').replace('  ', ' '))
            all_sent_rouge_scores.append(rouge_scores)
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # replace bos_token_id at the begining of document with bosent_token_id
    for i in range(len(model_inputs['input_ids'])):
        model_inputs['input_ids'][i][0] = bosent_token_id

    all_token_sent_id = []
    for sent_tokens in model_inputs['input_ids']:
        sid = -1
        token_sent_id = []
        for tid in sent_tokens:
            if tid == bosent_token_id:
                sid += 1
            if tid == tokenizer.eos_token_id or tid == tokenizer.pad_token_id:
                sid = -1
            token_sent_id.append(sid)
        all_token_sent_id.append(token_sent_id)

    all_token_info_dist = []
    all_sent_bos_idx = []
    for token_sent_id, sent_rouge_scores in zip(all_token_sent_id, all_sent_rouge_scores):
        sent_rouge_scores = sent_rouge_scores[:max(token_sent_id) + 1]  # truncation
        sent_bos_idx = []
        token_info_dist = []
        bos_idx = 0
        for sid in range(max(token_sent_id) + 1):
            tnum = token_sent_id.count(sid)
            sent_score = sent_rouge_scores[sid]
            token_info_dist.extend([sent_score for _ in range(tnum)])
            sent_bos_idx.extend([bos_idx for _ in range(tnum)])
            bos_idx += tnum
        token_info_dist.extend([-1 for _ in range(token_sent_id.count(-1))])
        all_token_info_dist.append(token_info_dist)
        sent_bos_idx.extend([0 for _ in range(token_sent_id.count(-1))])
        all_sent_bos_idx.append(sent_bos_idx)

    for i in range(len(all_token_sent_id)):
        for j in range(len(all_token_sent_id[i])):
            all_token_sent_id[i][j] += 1

    model_inputs['info_distribution'] = all_token_info_dist
    model_inputs['sentence_bos_index'] = all_sent_bos_idx
    model_inputs['sent_id'] = all_token_sent_id

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

convert_to_features = partial(
    convert_to_features,
    tokenizer=tokenizer,
    padding='max_length',
    max_source_length=max_source_length,
    max_target_length=max_target_length,
    src_text_column_name=src_text_column_name,
    tgt_text_column_name=tgt_text_column_name,
)
dataset = dataset.map(
    convert_to_features,
    batched=True,
    num_proc=n_proc,
)

cols_to_keep = ["input_ids", "attention_mask", "labels", "info_distribution", "sentence_bos_index", "sent_id"]
dataset.set_format(columns=cols_to_keep)

for split in ['train', 'validation', 'test']:
    with open(f'data/{split}.json', 'w') as outfile:
        for i, example in enumerate(dataset[split]):
            json_string = json.dumps(example)
            outfile.write(json_string + '\n')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


...

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


#38:   0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


#39:   0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [6]:
!python run.py \
    --model_name_or_path facebook/bart-large-cnn \
    --do_train \
    --do_predict \
    --train_file data/train.json \
    --validation_file data/validation.json \
    --test_file data/test.json \
    --output_dir outputs/train \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 2 \
    --per_device_eval_batch_size 4 \
    --num_train_epochs 10 \
    --learning_rate 1.3214238628872776e-05 \
    --warmup_steps 1500 \
    --weight_decay 0.01 \
    --max_grad_norm 0.1 \
    --metric_for_best_model rougeLsum \
    --evaluation_strategy epoch \
    --save_strategy epoch \
    --save_total_limit 2\
    --load_best_model_at_end True\
    --fp16 true \
    --bosent_token_id 50264 \
    --encoder_loss_ratio 1.0 \
    --encoder_label_smoothing 0.1 \
    --encoder_label_smoothing_type adjacent \
    --lower_saliency_threshold 0.125 \
    --higher_saliency_threshold 0.230 \
    --marginal_distribution true \
    --marginal_temperature 0.5 \
    --num_beams 5 \
    --max_length 128 \
    --min_length 20 \
    --length_penalty 1.5 \
    --no_repeat_ngram_size 3 \
    --overwrite_output_dir \
    --predict_with_generate


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [1]:
%mkdir season_samsum

mkdir: cannot create directory ‘season_samsum’: File exists


In [3]:
%cp -av /kaggle/working/outputs/train/pytorch_model.bin season_samsum
%cp -av /kaggle/working/outputs/train/config.json season_samsum

'/kaggle/working/outputs/train/pytorch_model.bin' -> 'season_samsum/pytorch_model.bin'
'/kaggle/working/outputs/train/config.json' -> 'season_samsum/config.json'


In [4]:
!zip -r season_samsum.zip season_samsum

  adding: season_samsum/ (stored 0%)
  adding: season_samsum/config.json (deflated 61%)
  adding: season_samsum/pytorch_model.bin (deflated 7%)


In [5]:
from IPython.display import FileLink
FileLink(r'season_samsum.zip')


/kaggle/working/season_samsum.zip